In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/final-merged-data/app_train_merge_bureau.csv
/kaggle/input/final-merged-data/final_merged_table.csv


In [2]:
final = pd.read_csv("../input/final-merged-data/final_merged_table.csv")

# SELECT CATEGORICAL FEATURES

In [3]:
from sklearn.preprocessing import OrdinalEncoder
def prepare_categorical(x):
    oe = OrdinalEncoder()
    oe.fit(x)
    x_new = oe.transform(x)
    return x_new

In [4]:
cat = final.select_dtypes(include = object)
encoded = prepare_categorical(final[cat.columns])
for i in range(len(cat.columns)):
    cat[cat.columns[i]] = encoded[: ,i]
cat.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE_x,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,...,CODE_REJECT_REASON,NAME_TYPE_SUITE_y,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,NAME_SELLER_INDUSTRY,NAME_YIELD_GROUP,PRODUCT_COMBINATION
0,0.0,1.0,0.0,1.0,6.0,7.0,4.0,3.0,1.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,1.0,4.0,1.0,1.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,1.0,1.0,1.0,6.0,7.0,4.0,3.0,1.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,0.0,1.0,6.0,7.0,4.0,0.0,1.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,1.0,0.0,1.0,6.0,7.0,4.0,3.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
target = final["TARGET"]
features = cat.fillna(cat.max() + 1, downcast='infer')
selector = SelectKBest(score_func=chi2, k=20)
selector.fit(features, target)
list(features.columns[selector.get_support()])

['NAME_CONTRACT_TYPE_x',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE_x',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START_x',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE',
 'WEEKDAY_APPR_PROCESS_START_y',
 'NAME_PRODUCT_TYPE',
 'CHANNEL_TYPE',
 'NAME_SELLER_INDUSTRY']

# SELECTING NUMERIC FEATURES

In [6]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

numeric = final.select_dtypes(include = np.number).drop(["Unnamed: 0", "SK_ID_CURR", "TARGET", "SK_ID_PREV", "DEBT_OVER_CREDIT", "SK_DPD_mean_pos", "SK_DPD_DEF_mean_pos"], axis = 1)
target = final["TARGET"]
features = numeric.fillna(numeric.mean(), downcast='infer').round(3)

selector = SelectKBest(score_func=f_classif, k=30)
selector.fit(features, target)
selected = list(features.columns[selector.get_support()])
selected

/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 79 110] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


['CNT_CHILDREN',
 'AMT_CREDIT',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'REGION_RATING_CLIENT',
 'HOUR_APPR_PROCESS_START_x',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'ELEVATORS_AVG',
 'FLOORSMAX_AVG',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'DAYS_LAST_PHONE_CHANGE',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_6',
 'AMT_REQ_CREDIT_BUREAU_YEAR',
 'DAYS_CREDIT_ENDDATE',
 'AMT_CREDIT_SUM',
 'DAYS_CREDIT_UPDATE']